In [17]:
from openai import OpenAI
import json
import os
import chardet
import re
import pandas as pd

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [18]:
def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."

def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [19]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [20]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [21]:
def create_directory_if_not_exists(path, directory_name):
    # Join the given path and directory name to form the full path
    full_path = os.path.join(path, directory_name)
    
    # Check if the directory exists
    if not os.path.exists(full_path):
        # Create the directory if it does not exist
        os.makedirs(full_path)
        print(f"Directory '{full_path}' created.")
    else:
        print(f"Directory '{full_path}' already exists.")

In [22]:
#Prompting list
MAIN_FLOW_SCENARIO_GENERATOR="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SCENARIO_GENERATOR="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

CONDITION_EXTRACTOR="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

CONDITION_SCENARIO_GENERATOR="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
SCENARIO_REFINER="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""
USECASE_ANALYZER ="""
Given use case specification flows.
Extract all the contents of flows into json format. 
If the use case's main scenario has a name other than 'Main flow', rename it to Main flow.
Other flows just keep the same name.
Skip all the information that not describe the flow action of use case(description, precondition,postcondition,...).
Return the flows in json format.
{
"Flow name":[ contents of corresponding flow],
}
For example: 
{
  "Main flow": [
    "Step 1: Learner fills in the username field by a valid username that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding username",
    "Step 3: Learner press \"Login\" button",
    "Step 4: System redirects learner to Home page"
  ],
  "Alternative flow 1: Login by email": [
    "At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding email",
    "Go back to step 3 in the basic flow and continue with the steps from step 3"
  ]
}
"""
TEST_CASE_GENERATOR="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""
TEST_CASE_VALIDATOR="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [23]:
usecase_path = r"D:\Dissertation-GPT\dataset\SpecificationData\test" #path to the directory store use case specification file
save_path = r"D:\Dissertation-GPT\evaluate\ResultSet\7.28\test" #path to the directory to save list of test scenario generated by method

In [24]:
scenario_save_path = os.path.join(save_path,"ts")
testcase_save_path = os.path.join(save_path,"tc")
create_directory_if_not_exists(save_path, "ts")
create_directory_if_not_exists(save_path, "tc")
create_directory_if_not_exists(testcase_save_path, "refined")
create_directory_if_not_exists(testcase_save_path, "raw")
testcase_refined_save_path = os.path.join(testcase_save_path,"refined")

Directory 'D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\ts' already exists.
Directory 'D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc' already exists.
Directory 'D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined' already exists.
Directory 'D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\raw' already exists.


In [25]:
usecase_name_list = [] # list of use case name in folder
project_name = os.path.basename(usecase_path)
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])

In [26]:
#Test Design Phase
for usecase_name in usecase_name_list:
    # print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    usecase_content = read_file_content(usecase_directlink) # store use case specification content

    #Use Case Analyzer
    promptUSECASE_ANALYZER = [
    { "role": "system", "content": USECASE_ANALYZER},
    { "role": "user", "content": usecase_content}
    ]
    gpt_response = askJSON(promptUSECASE_ANALYZER, client, model)
    flows = json.loads(gpt_response)
    #get the response flow into parameters
    main_flow = flows['Main flow']
    alt_prompt=[]
    main_flow_prompt = "Main flow:"
    for i in main_flow:
        main_flow_prompt+="\n"+i
    for key,value in flows.items():
        if(key not in 'Main flow'):
            flow = key+ ":" 
            for i in value:
                flow += "\n"+ i
            alt_prompt.append(flow)

    #Condition Extractor
    promptCONDITION_EXTRACTOR = [
        { "role": "system", "content": CONDITION_EXTRACTOR},
        { "role": "user", "content": usecase_content}
        ]
    gpt_response = askJSON(promptCONDITION_EXTRACTOR, client, model)
    full_elements = json.loads(gpt_response)
    #filter to get condition of Text Field only
    condition_element = {key: value for key, value in full_elements.items() 
                                    if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
    print(condition_element)

    scenario_list = ""

    #Condition Scenario Generator
    if(len(condition_element)!= 0):
        promptCONDITION_SCENARIO_GENERATOR = [
            { "role": "system", "content": CONDITION_SCENARIO_GENERATOR},
            { "role": "user", "content": usecase_content + '\nElement:' + str(condition_element)}
        ]
        condition_scenario_response = ask(promptCONDITION_SCENARIO_GENERATOR, client, model)
        scenario_list = "\nCondition Scenario: " + condition_scenario_response
        print("\nCondition Scenario Generator: \n"+condition_scenario_response)

    #Main Flow Scenario Generator
    promptMAIN_FLOW_SCENARIO_GENERATOR = [
        { "role": "system", "content": MAIN_FLOW_SCENARIO_GENERATOR},
        { "role": "user", "content": main_flow_prompt}
        ]
    main_gpt_response = ask(promptMAIN_FLOW_SCENARIO_GENERATOR, client, model)
    scenario_list += "\n Flow cover scenarios: " +  main_gpt_response
    print(main_flow_prompt)
    print("\nMain Flow Scenario Generator: \n"+main_gpt_response+"\n")

    #Sub Flow Scenario Generator
    for alt in alt_prompt:
        promptSUB_FLOW_SCENARIO_GENERATOR = [
            { "role": "system", "content": SUB_FLOW_SCENARIO_GENERATOR},
            { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
        ]
        sub_gpt_response = ask(promptSUB_FLOW_SCENARIO_GENERATOR, client, model)
        print(alt)
        print("\nSub Flow Scenario Generator:"+sub_gpt_response)
        scenario_list+=  "\n" + sub_gpt_response
    
    scenario_refiner_input = "Use case: " + usecase_content +"\n Test scenarios: \n"
    scenario_refiner_input += scenario_list

    #Scenario Refiner
    promptSCENARIO_REFINER = [
        { "role": "system", "content": SCENARIO_REFINER},
        { "role": "user", "content": scenario_refiner_input}
        ]
    filter_gpt_response = ask(promptSCENARIO_REFINER, client, model)
    print("Scenario Refiner Response: \n" +filter_gpt_response)

    write_string_to_file(os.path.join(scenario_save_path,f"{usecase_name}.txt"), filter_gpt_response) #write scenario to save path directory

In [27]:
#Test Case Generation Phase

for usecase_name in usecase_name_list:
    # print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    usecase_content = read_file_content(usecase_directlink) # store use case specification content
    testscenario_path = os.path.join (scenario_save_path,f"{usecase_name}.txt" )
    testscenario_list = read_file_to_list(testscenario_path)

    for testscenario in testscenario_list:
        content = "Test scenarios:"+ testscenario+  "\nUse case:" + usecase_content
        print(content)
        testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
        testcase_path = os.path.join (testcase_save_path,f"refined/{usecase_name}-{testscenario}.txt" )
        testcase_raw_path = os.path.join (testcase_save_path,f"raw/{usecase_name}-{testscenario}.txt" )
        promptTEST_CASE_GENERATOR = [
        { "role": "system", "content": TEST_CASE_GENERATOR},
        { "role": "user", "content": content}
        ]
        gpt_response = askJSON(promptTEST_CASE_GENERATOR, client, model)
        print(gpt_response)
        json_data = json.loads(gpt_response)
        write_json_to_txt_raw(json_data, testcase_raw_path)

        promptTEST_CASE_VALIDATOR = [
            { "role": "system", "content": TEST_CASE_VALIDATOR},
            { "role": "user", "content": "Test scenario: "+ testscenario + "\n" + gpt_response}
        ]
        final_response = askJSON(promptTEST_CASE_VALIDATOR, client, model)
        json_fin = json.loads(final_response)
        print(final_response)
        write_json_to_txt(json_fin, testcase_path)
    print("f{usecase_name} gen test case complete")

In [28]:
#Print results as excel
def split_before_and_after_hyphen(file_name):
    parts = file_name.split('-', 1)
    if len(parts) == 2:
        before_hyphen, after_hyphen = parts
    else:
        before_hyphen, after_hyphen = file_name, ""
    return before_hyphen.strip(), after_hyphen.strip()

# Function to parse the test cases from the text file
def parse_test_cases(file_name):
    # with open(file_name, 'r') as file:
    #     content = file.read()
    content = read_file_content(file_name)
    base_name = os.path.basename(file_name)

    base_nam = os.path.splitext(base_name)[0]

    # Extract use case and test scenario names from the file name
    use_case_name, test_scenario_name = split_before_and_after_hyphen(base_nam)
    
    # Regular expression to extract test cases
    test_case_pattern = re.compile(
        r"Test Case: (.*?)\ntestCaseName: (.*?)\nobjective: (.*?)\ntestSteps:\n(.*?)\nexpectedResult: (.*?)\n",
        re.DOTALL
    )
    
    matches = test_case_pattern.findall(content)
    
    test_cases = []
    for match in matches:
        test_case_name = match[1].strip()
        objective = match[2].strip()
        test_steps = match[3].strip().replace('\n', ' ')
        expected_result = match[4].strip()
        
        test_cases.append({
            'Use Case Name': use_case_name,
            'Test Scenario Name': test_scenario_name,
            'Test Case Name': test_case_name,
            'Objective': objective,
            'Test Steps': test_steps,
            'Expected Result': expected_result
        })
    
    return test_cases

# Function to write the parsed test cases to an Excel file
def write_to_excel(test_cases, output_file):
    df = pd.DataFrame(test_cases)
    df.to_excel(output_file, index=False)
all_test_cases = []

output_file = os.path.join(save_path,'test_cases.xlsx') 
# Parse the test cases and write to Excel
file_list = os.listdir(testcase_refined_save_path)

def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

# Function to sort filenames naturally by the extracted number
def natural_sort_key(filename):
    # Split the filename into parts: prefix and number
    parts = re.split(r'(\d+)', filename)
    # Convert numerical parts to integers for proper numeric sorting
    parts[1::2] = map(int, parts[1::2])
    return parts

# Sort filenames using the natural sort key
sorted_files = sorted(file_list, key=natural_sort_key)
for filename in sorted_files:
    file_path = os.path.join(testcase_refined_save_path, filename)
    print(file_path)
    if os.path.isfile(file_path):
        test_cases = parse_test_cases(file_path)
        # test_cases = parse_test_cases(file_path)
        all_test_cases.extend(test_cases)
    
print(all_test_cases)
write_to_excel(all_test_cases, output_file)

D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\Remove the book-1. Successful Book Deletion Scenario.txt
D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\add to cart-1. Successful Product Addition to Cart Scenario.txt
D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\add to cart-2. User chooses a product option before adding to cart.txt
D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\add to cart-3. User adjusts quantity before adding product to cart.txt
D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\add to cart-4. Attempt to add a product with multiple options without selecting an option.txt
D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\add to cart-5. Attempt to add a product with a quantity greater than the current stock.txt
D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\add to cart-6. Attempt to add a product that is out of stock.txt
D:\Dissertation-GPT\evaluate\ResultSet\7.28\test\tc\refined\add to 